In [ ]:
%load_ext lab_black

Step: Load data

In [ ]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

In [ ]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [ ]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [ ]:
import os

script_path = os.path.abspath("production_injury_data.py")

src = ScriptRunConfig(source_directory=".", script=script_path, environment=environment)

In [69]:
# Create a ScriptRunConfig for your script
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

In [70]:
# Create a PythonScriptStep
step = PythonScriptStep(
    name="scrapper_nfl_injury",
    script_name="production_injury_data.py",
    compute_target=compute_target,
    runconfig=src.run_config,
)

In [71]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [72]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="scrapper_nfl_injury_pipeline")

Created step scrapper_nfl_injury [e5ed55fc][00b2cd3a-7d2a-4d1c-ab4e-b62ba442fe3e], (This step will run and generate new outputs)


In [73]:
from datetime import time

recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="15:31")
schedule = Schedule.create(
    workspace,
    name="scrapper_nfl_injury_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="scrapper_nfl_injury_experiment",
    recurrence=recurrence,
)

In [74]:
schedules = Schedule.list(workspace)
for s in schedules:
    print(f"Schedule name: {s.name}, Status: {s.status}, Pipeline ID: {s.pipeline_id}")

Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: 251973e9-230c-45f9-b49f-a1761de800e7
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: cccbad01-989d-43d2-861c-b77071147192
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: d7e805cc-941e-4477-bff1-c6a4ad480fa6
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: bfc408c3-fa3b-4702-9365-89291d9c0a76
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: fadf4d78-69b0-49cf-8934-5a0e1a6e0202
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: fadf4d78-69b0-49cf-8934-5a0e1a6e0202
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: fadf4d78-69b0-49cf-8934-5a0e1a6e0202
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: fadf4d78-69b0-49cf-8934-5a0e1a6e0202
Schedule name: scrapper_nfl_injury_schedule, Status: Active, Pipeline ID: b38cc710-8776-4f27-b176-97fa6205fb18
S